In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
import matplotlib
import matplotlib.pyplot as plt

In [2]:
plt.style.use('ggplot')
matplotlib.rcParams['figure.dpi'] = 100

## Feature Engineering

In [3]:
path = ''
wh = pd.read_csv(path+'wh_cleaned.csv', memory_map=True)
perf = pd.read_csv(path+'perf_cleaned.csv', memory_map=True)
prod = pd.read_csv(path+'prod_cleaned2.csv', memory_map=True)

C:\Users\Nguyen\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Assign correct dtypes
for var in wh.columns[wh.columns.str.contains('Date')].values: 
    wh[var] = pd.to_datetime(wh[var])
wh['CurrentOperatorParent'] = wh['CurrentOperatorParent'].astype('object')
wh['LicenseeParentCompany'] = wh['LicenseeParentCompany'].astype('object')

perf['ActivityDate'] = pd.to_datetime(perf['ActivityDate'])
perf['ObservationNumber'] = perf['ObservationNumber'].astype('int64')
perf['PerfShots'] = perf['PerfShots'].astype('int64')

prod['ProdPeriod'] = pd.to_datetime(prod['ProdPeriod'])

In [5]:
datatype = pd.DataFrame(wh.dtypes, columns=['pdtype'])
datatype['nunique'] = wh.nunique()
datatype['nnan'] = wh.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8694,0
Province,object,2,0
LicenceNumber,object,8687,0
UWI,object,8694,0
CurrentOperator,object,106,0
CurrentOperatorParent,object,104,0
CurrentOperatorParentName,object,104,0
Licensee,object,103,0
LicenseeParentCompany,object,103,0
LicenseeParentCompanyName,object,103,0


In [6]:
datatype = pd.DataFrame(perf.dtypes, columns=['pdtype'])
datatype['nunique'] = perf.nunique()
datatype['nnan'] = perf.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8415,0
EPAPTId,int64,376653,0
ObservationNumber,int64,686,0
ActivityDate,datetime64[ns],1830,0
ActivityType,object,18,0
IntervalTop,float64,52870,0
IntervalBase,float64,54848,0
PerfShots,int64,29,0


In [7]:
datatype = pd.DataFrame(prod.dtypes, columns=['pdtype'])
datatype['nunique'] = prod.nunique()
datatype['nnan'] = prod.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8661,0
ProdPeriod,datetime64[ns],61,0
Condensate,float64,10212,0
Duration,float64,744,0
Gas,float64,27296,0
Oil,float64,11585,0
Water,float64,13556,0


### perf

In [8]:
perf_agg = pd.DataFrame()

In [9]:
comlist = np.setdiff1d(perf.ActivityType.unique(), ['Bridge Plug - No Cement', 'Cement Plug', 'Cement Squeeze', 
                                                    'Packing Device Capped w/Cement', 'Remedial Casing Cementing'])
perf_agg['CompletionTop_perf'] = perf[perf.ActivityType.isin(comlist)].groupby('EPAssetsId')['IntervalTop'].min()
perf_agg['CompletionEnd_perf'] = perf[perf.ActivityType.isin(comlist)].groupby('EPAssetsId')['IntervalBase'].max()

perf_agg['CompletionStartDate_perf'] = perf.groupby('EPAssetsId')['ActivityDate'].min()
perf_agg['CompletionEndDate_perf'] = perf.groupby('EPAssetsId')['ActivityDate'].max()
perf_agg['CompletionTime_perf'] = (perf_agg['CompletionEndDate_perf'] - perf_agg['CompletionStartDate_perf']).dt.days

perf_agg['nCompletion_perf'] = perf.groupby('EPAssetsId')['ObservationNumber'].nunique()
perf_agg['nCompletionType_perf'] = perf.groupby('EPAssetsId')['ActivityType'].nunique()

In [10]:
perf_agg.reset_index(inplace=True)
perf_agg.head()

,EPAssetsId,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf
0,1145472,835.0,1510.5,2015-02-22,2015-02-22,0,1,1
1,1145481,913.0,1480.5,2015-01-22,2015-01-22,0,19,1
2,1145482,896.0,1492.5,2015-02-15,2015-02-15,0,1,1
3,1145487,870.0,1447.0,2015-02-22,2015-02-22,0,1,1
4,1145496,925.0,1490.5,2015-01-21,2015-01-21,0,19,1


### prod

In [11]:
prod_agg = pd.DataFrame()

In [12]:
for var in ['Condensate', 'Gas', 'Oil', 'Water']: 
    prod[var+'Rate_prod'] = prod[var]/prod['Duration']*24

In [13]:
prod_agg['StartDate_prod'] = prod.groupby('EPAssetsId')['ProdPeriod'].min() + pd.DateOffset(days=1, months=-1)
prod_agg['CumDuration_prod'] = prod.groupby('EPAssetsId')['Duration'].sum()
prod_agg['CumProdPeriod_prod'] = prod.groupby('EPAssetsId')['ProdPeriod'].apply(lambda x: (max(x)-min(x)))
prod_agg['CumProdPeriod_prod'] = prod_agg['CumProdPeriod_prod'].dt.days

for var in ['Condensate', 'Gas', 'Oil', 'Water']: 
    prod_agg['Cum'+var+'_prod'] = prod.groupby('EPAssetsId')[var].sum()
    prod_agg['Avg'+var+'_prod'] = prod_agg['Cum'+var+'_prod']/prod_agg['CumDuration_prod']*24
    prod_agg['Max'+var+'_prod'] = prod.groupby('EPAssetsId')[var+'Rate_prod'].max()
    prod_agg['Min'+var+'_prod'] = prod.groupby('EPAssetsId')[var+'Rate_prod'].min()

In [14]:
prod_agg.reset_index(inplace=True)
prod_agg.head()

,EPAssetsId,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod
0,1145472,2015-03-01,42942.0,1767,0.0,0.0,0.0,0.0,8.585028,0.004798,0.019936,0.000716,15956.59362,8.918035,42.790930,0.713977,3349.31850,1.871912,13.249159,0.000000
1,1145481,2015-01-29,40768.0,1767,0.0,0.0,0.0,0.0,7.108869,0.004185,0.027289,0.000228,8735.27424,5.142430,42.141660,0.040579,4367.63712,2.571215,33.793380,0.202897
2,1145482,2015-03-01,42817.0,1767,0.0,0.0,0.0,0.0,7.737473,0.004337,0.018952,0.000631,8817.67062,4.942525,23.314192,1.379698,2815.31448,1.578054,42.770640,0.304345
3,1145487,2015-01-29,42855.0,1798,0.0,0.0,0.0,0.0,6.300160,0.003528,0.018875,0.000342,14170.91940,7.936112,35.374703,1.299892,2829.78102,1.584757,184.920120,0.466663
4,1145496,2015-01-29,43114.0,1798,0.0,0.0,0.0,0.0,15.948164,0.008878,0.018329,0.005000,7876.71654,4.384682,18.450080,0.464898,1522.76058,0.847666,6.084851,0.121738


### Joining

In [17]:
main = pd.concat([df.set_index('EPAssetsId') for df in [wh, perf_agg, prod_agg]], axis=1, join='outer').reset_index()
main.head()

,EPAssetsId,Province,LicenceNumber,UWI,CurrentOperator,CurrentOperatorParent,CurrentOperatorParentName,Licensee,LicenseeParentCompany,LicenseeParentCompanyName,LicenceDate,CurrentStatus,CurrentStatusStandardised,WellType,WellTypeStandardised,ConfidentialReleaseDate,WellName,Formation,Field,Pool,Surf_Location,Surf_Township,Surf_Meridian,Surf_Range,Surf_Section,Surf_LSD,Surf_Longitude,Surf_Latitude,Surf_TownshipRange,Surf_QuarterSection,BH_Location,BH_TownshipRange,BH_QuarterSection,BH_Longitude,BH_Latitude,BH_Township,BH_Meridian,BH_Range,BH_Section,BH_LSD,GroundElevation,KBElevation,TotalDepth,LaheeClass,Confidential,SurfaceOwner,DrillingContractor,SpudDate,FinalDrillDate,RigReleaseDate,DaysDrilling,DrillMetresPerDay,TVD,WellProfile,RegulatoryAgency,PSACAreaCode,PSACAreaName,ProjectedDepth,StatusDate,StatusSource,UnitID,UnitName,UnitFlag,Municipality,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod
0,1145472,Saskatchewan,14A342,193120603021W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2014-01-24,Active,Active,Oil Production,Oil,2015-03-16,CPEC AVON HILL HZ 11A10-6-5C12-6-30-21,Viking,Avon Hill,Avon Hill Viking,10-06-030-21W3,30,W3,21,6,10,-108.942475,51.542038,030-21-W3,NE,12-06-030-21-W3,030-21-W3,NW,-108.955296,51.542254,30,W3,21,6,12,691.2,695.7,1518.0,Development,Non-Confidential,Unspecified,Precision Drilling Corporation,2015-01-16,2015-02-14,2015-02-15,29,52.344828,NaN,Horizontal,SKER,SK2,Southwestern Saskatchewan,705.0,2014-01-24,SKER,Unspecified,Unspecified,No,Kindersley,835.0,1510.5,2015-02-22,2015-02-22,0.0,1.0,1.0,2015-03-01,42942.0,1767.0,0.0,0.0,0.0,0.0,8.585028,0.004798,0.019936,0.000716,15956.59362,8.918035,42.790930,0.713977,3349.31850,1.871912,13.249159,0.000000
1,1145481,Saskatchewan,14B018,193083202922W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2014-03-02,Active,Active,Oil Production,Oil,2016-01-18,CPEC AVON HILL HZ 11A6-32-2D8-32-29-22,Viking,Avon Hill,Avon Hill Viking,06-32-029-22W3,29,W3,22,32,6,-109.065286,51.523909,029-22-W3,SW,08-32-029-22-W3,029-22-W3,SE,-109.052504,51.524055,29,W3,22,32,8,712.2,716.7,1504.0,Development,Non-Confidential,Unspecified,Precision Drilling Corporation,2015-01-16,2015-01-18,2015-01-19,2,752.000000,738.3,Horizontal,SKER,SK2,Southwestern Saskatchewan,724.0,2014-03-02,SKER,Unspecified,Unspecified,No,Kindersley,913.0,1480.5,2015-01-22,2015-01-22,0.0,19.0,1.0,2015-01-29,40768.0,1767.0,0.0,0.0,0.0,0.0,7.108869,0.004185,0.027289,0.000228,8735.27424,5.142430,42.141660,0.040579,4367.63712,2.571215,33.793380,0.202897
2,1145482,Saskatchewan,14B020,193093402922W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2014-03-02,Active,Active,Oil Production,Oil,2016-01-30,CPEC AVON HILL HZ 3A11-34-6D9-34-29-22,Viking,Avon Hill,Avon Hill Viking,11-34-029-22W3,29,W3,22,34,11,-109.018393,51.527626,029-22-W3,NW,09-34-029-22-W3,029-22-W3,NE,-109.005705,51.527585,29,W3,22,34,9,703.4,707.9,1503.0,Development,Non-Confidential,Unspecified,Precision Drilling Corporation,2015-10-01,2015-01-30,2015-01-31,20,75.150000,735.6,Horizontal,SKER,SK2,Southwestern Saskatchewan,737.0,2014-03-02,SKER,Unspecified,Unspecified,No,Kindersley,896.0,1492.5,2015-02-15,2015-02-15,0.0,1.0,1.0,2015-03-01,42817.0,1767.0,0.0,0.0,0.0,0.0,7.737473,0.004337,0.018952,0.000631,8817.67062,4.942525,23.314192,1.379698,2815.31448,1.578054,42.770640,0.304345
3,1145487,Saskatchewan,14B015,193093602922W300,Crescent Point Re

In [18]:
# Sanity check if any vertical wells are mislabelled
print(main[(main.TotalDepth==main.TVD)&(main.WellProfile!='Vertical')].EPAssetsId)
print(main[(main.Surf_Latitude==main.BH_Latitude)&(main.Surf_Latitude==main.BH_Latitude)&(main.WellProfile!='Vertical')].EPAssetsId)

4348    2585178
Name: EPAssetsId, dtype: int64
4348    2585178
Name: EPAssetsId, dtype: int64


In [19]:
# Fix this well profile
main.loc[main.EPAssetsId==2585178, 'WellProfile'] = 'Vertical'

In [20]:
# Drop wells with questionable TVD
main = main[main.EPAssetsId.isin([1151263, 1152677, 1151511, 1151468, 1173448])==False].reset_index(drop=True)

In [21]:
# nnan for _perf and _prod are expected to 279 and 33
datatype = pd.DataFrame(main.dtypes, columns=['pdtype'])
datatype['nunique'] = main.nunique()
datatype['nnan'] = main.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8689,0
Province,object,2,0
LicenceNumber,object,8682,0
UWI,object,8689,0
CurrentOperator,object,106,0
CurrentOperatorParent,object,104,0
CurrentOperatorParentName,object,104,0
Licensee,object,103,0
LicenseeParentCompany,object,103,0
LicenseeParentCompanyName,object,103,0


### Feature Generation

#### Functions

In [24]:
def EarthRadius(lat1, lat2, Re=6378.14e3, Rp=6356.75e3): 
    '''Calculate Earth radius (m) as a function of latitudes'''
    lat = (lat1 + lat2)/2
    lat = np.deg2rad(lat)
    R = np.sqrt(((Re**2*np.cos(lat))**2 + (Rp**2*np.sin(lat))**2)/((Re*np.cos(lat))**2 + (Rp*np.sin(lat))**2))
    return R

In [25]:
def HaversineDistance(lat1, lon1, lat2, lon2): 
    '''Calculate great-circle distance (m)'''
    lat1, lon1 = np.deg2rad(lat1), np.deg2rad(lon1)
    lat2, lon2 = np.deg2rad(lat2), np.deg2rad(lon2)
    hvs = np.sin((lat2-lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2-lon1)/2)**2
    theta = np.arcsin(np.sqrt(hvs))
#     R = 6371e3
    R = EarthRadius(lat1, lat2)
    return 2*R*theta

In [26]:
def Azimuth(lat1, lon1, lat2, lon2): 
    '''Calculate azimuth (degree from north)
    Start point should be surface lat and lon'''
    lat1, lon1 = np.deg2rad(lat1), np.deg2rad(lon1)
    lat2, lon2 = np.deg2rad(lat2), np.deg2rad(lon2)
    x = np.sin(lon2 - lon1)*np.cos(lat2)
    y = np.cos(lat1)*np.sin(lat2) - (np.sin(lat1)*np.cos(lat2)*np.cos(lon2-lon1))
    azimuth = np.arctan2(x, y)
    return np.rad2deg(azimuth)

In [29]:
main_add = main.copy(deep=True)

#### Numerical

##### Distance Matrices

In [30]:
# Geo distance matrix
# Haversine takes too long, use Euclidean instead
from scipy.spatial.distance import cdist
df = main_add[['Surf_Latitude', 'Surf_Longitude', 'BH_Latitude', 'BH_Longitude']].copy(deep=True).values
distance_matrix = cdist(df, df, metric='euclidean')
print(np.mean(distance_matrix))
print(np.min(distance_matrix))
print(np.max(distance_matrix))

6.787474504859801
0.0
18.567052397196566


In [31]:
# Well profile distance matrix
df = main_add['WellProfile'].factorize()[0].reshape(-1,1)
wellprofile_matrix = cdist(df, df, metric='euclidean')
print(np.mean(wellprofile_matrix))
print(np.min(wellprofile_matrix))
print(np.max(wellprofile_matrix))

0.6602027072750014
0.0
2.0


##### FillNA perf with neighor

In [32]:
# For the purpose of fillna perf and prod, wellprofile is needed
totaldistance_matrix = distance_matrix + wellprofile_matrix*1000

In [33]:
# train+test x train+test
n_neighbors = 50
neighbors = np.empty((totaldistance_matrix.shape[0], n_neighbors))
for i in range(0, totaldistance_matrix.shape[0]): 
    neighbors[i, :] = list(main_add.EPAssetsId.values[np.argsort(totaldistance_matrix[i, :])[1:n_neighbors+1]])

In [34]:
# fillna CompletionTop_perf, CompletionEnd_perf, LateralLength_pewh (x2)
# fillna for other perf variables by the way
for i in range(2): 
    for index in main_add[main_add.CompletionTop_perf.isnull()].index: 
        mask = main_add.EPAssetsId.isin(neighbors[index, :10])
        main_add.loc[main_add.index==index, 'CompletionTop_perf'] = main_add.loc[mask, 'CompletionTop_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionEnd_perf'] = main_add.loc[mask, 'CompletionEnd_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionStartDate_perf'] = main_add.loc[mask, 'CompletionStartDate_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionEndDate_perf'] = main_add.loc[mask, 'CompletionEndDate_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionTime_perf'] = main_add.loc[mask, 'CompletionTime_perf'].mean()
        main_add.loc[main_add.index==index, 'nCompletion_perf'] = main_add.loc[mask, 'nCompletion_perf'].mean()
        main_add.loc[main_add.index==index, 'nCompletionType_perf'] = main_add.loc[mask, 'nCompletionType_perf'].mean()
    print(main_add.CompletionTop_perf.isnull().sum())
    print(main_add.CompletionEnd_perf.isnull().sum())
    print(main_add.CompletionStartDate_perf.isnull().sum())
    print(main_add.CompletionEndDate_perf.isnull().sum())
    print(main_add.CompletionTime_perf.isnull().sum())
    print(main_add.nCompletion_perf.isnull().sum())
    print(main_add.nCompletionType_perf.isnull().sum())

1
1
1
1
1
1
1
0
0
0
0
0
0
0


In [36]:
# Generate neighbor rates & volumes features
varlist = main_add.columns[main_add.columns.str.contains('_prod')]
for var in varlist:
    for index in main_add[main_add[var].isnull()].index:
        mask = main_add.EPAssetsId.isin(neighbors[index, :10])
        main_add.loc[main_add.index == index, var] = main_add.loc[mask, var].mean()
        main_add.loc[main_add.index == index, var] = main_add.loc[mask, var].min()
        main_add.loc[main_add.index == index, var] = main_add.loc[mask, var].max()
    print(main_add[var].isnull().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


##### TVD_calc

In [37]:
# Lateral length 
main_add['LateralLength_pewh'] = np.max([main_add.TotalDepth - main_add.CompletionTop_perf, 
                                         main_add.CompletionEnd_perf - main_add.CompletionTop_perf], axis=0)
# Azimuth
main_add['Azimuth_ltln'] = Azimuth(main_add.Surf_Latitude, main_add.Surf_Longitude,
                                   main_add.BH_Latitude, main_add.BH_Longitude)
# Horizontal Displacement
main_add['HorizontalDisplacement_hvs'] = HaversineDistance(main_add.Surf_Latitude, main_add.Surf_Longitude,
                                                           main_add.BH_Latitude, main_add.BH_Longitude)

In [38]:
# Sanity check
main_add[main_add.HorizontalDisplacement_hvs>(main_add.TotalDepth**2-main_add.TVD**2)**0.5]

,EPAssetsId,Province,LicenceNumber,UWI,CurrentOperator,CurrentOperatorParent,CurrentOperatorParentName,Licensee,LicenseeParentCompany,LicenseeParentCompanyName,LicenceDate,CurrentStatus,CurrentStatusStandardised,WellType,WellTypeStandardised,ConfidentialReleaseDate,WellName,Formation,Field,Pool,Surf_Location,Surf_Township,Surf_Meridian,Surf_Range,Surf_Section,Surf_LSD,Surf_Longitude,Surf_Latitude,Surf_TownshipRange,Surf_QuarterSection,BH_Location,BH_TownshipRange,BH_QuarterSection,BH_Longitude,BH_Latitude,BH_Township,BH_Meridian,BH_Range,BH_Section,BH_LSD,GroundElevation,KBElevation,TotalDepth,LaheeClass,Confidential,SurfaceOwner,DrillingContractor,SpudDate,FinalDrillDate,RigReleaseDate,DaysDrilling,DrillMetresPerDay,TVD,WellProfile,RegulatoryAgency,PSACAreaCode,PSACAreaName,ProjectedDepth,StatusDate,StatusSource,UnitID,UnitName,UnitFlag,Municipality,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod,LateralLength_pewh,Azimuth_ltln,HorizontalDisplacement_hvs
1732,1163751,Saskatchewan,68067,102161503223W300,Teine Energy Ltd.,1576,Teine Energy Ltd.,Teine Energy Ltd.,1576,Teine Energy Ltd.,2017-01-20,Active,Active,Oil Production,Oil,2017-03-18,TEINE HZ 16-15-32-23,Viking,Kerrobert,Kerrobert Viking,08-15-032-23W3,32,W3,23,15,8,-109.1755,51.741222,032-23-W3,SE,16-15-032-23-W3,032-23-W3,NE,-109.17484,51.749925,32,W3,23,15,16,702.3,706.6,888.0,Development,Non-Confidential,Unspecified,Savanna Drilling Corp.,2017-06-02,2017-02-15,2017-02-15,2,444.0,698.11,Horizontal,SKER,SK2,Southwestern Saskatchewan,721.0,2017-01-20,SKER,Unspecified,Unspecified,No,Oakdale,959.0,1495.5,2017-02-28,2017-02-28,0.0,1.0,1.0,2017-03-01,25338.0,1036.0,0.0,0.0,0.0,0.0,30.6144,0.028998,0.082477,0.003178,22717.49964,21.517878,57.174282,4.892454,1732.21092,1.64074,3.855039,0.0,536.5,2.688109,969.8561


In [39]:
# Fix the well's TotalDepth as this might be a typo (take CompletionEnd_perf as TD)
mask = (main_add.HorizontalDisplacement_hvs>(main_add.TotalDepth**2-main_add.TVD**2)**0.5) & (main_add.WellProfile=='Horizontal')
main_add.loc[mask, 'TotalDepth'] = main_add.loc[mask, 'CompletionEnd_perf']

In [40]:
# Another sanity check
main_add[main_add.HorizontalDisplacement_hvs<=main_add.TotalDepth-main_add.TVD*np.pi/2]

,EPAssetsId,Province,LicenceNumber,UWI,CurrentOperator,CurrentOperatorParent,CurrentOperatorParentName,Licensee,LicenseeParentCompany,LicenseeParentCompanyName,LicenceDate,CurrentStatus,CurrentStatusStandardised,WellType,WellTypeStandardised,ConfidentialReleaseDate,WellName,Formation,Field,Pool,Surf_Location,Surf_Township,Surf_Meridian,Surf_Range,Surf_Section,Surf_LSD,Surf_Longitude,Surf_Latitude,Surf_TownshipRange,Surf_QuarterSection,BH_Location,BH_TownshipRange,BH_QuarterSection,BH_Longitude,BH_Latitude,BH_Township,BH_Meridian,BH_Range,BH_Section,BH_LSD,GroundElevation,KBElevation,TotalDepth,LaheeClass,Confidential,SurfaceOwner,DrillingContractor,SpudDate,FinalDrillDate,RigReleaseDate,DaysDrilling,DrillMetresPerDay,TVD,WellProfile,RegulatoryAgency,PSACAreaCode,PSACAreaName,ProjectedDepth,StatusDate,StatusSource,UnitID,UnitName,UnitFlag,Municipality,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod,LateralLength_pewh,Azimuth_ltln,HorizontalDisplacement_hvs
3309,1170677,Saskatchewan,97847,102070103119W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2018-10-07,Active,Active,Oil Production,Oil,2018-12-28,CPEC HZ 7-1-31-19,Viking,Dodsland,Dodsland Viking,08-02-031-19W3,31,W3,19,2,8,-108.58093,51.626115,031-19-W3,SE,07-01-031-19-W3,031-19-W3,SE,-108.56257,51.624438,31,W3,19,1,7,707.6,711.8,1990.0,Development,Non-Confidential,Unspecified,Savanna Drilling Corp.,2018-11-25,2018-11-27,2018-11-28,2,995.0,134.0,Horizontal,SKER,SK2,Southwestern Saskatchewan,730.0,2018-10-07,SKER,Unspecified,Unspecified,No,Winslow,915.0,1971.5,2018-12-13,2018-12-13,0.0,1.0,1.0,2018-12-01,9792.0,396.0,0.0,0.0,0.0,0.0,11.09591,0.027196,0.082738,0.002051,15594.30114,38.221326,72.422554,7.952104,3843.69678,9.420825,73.950077,3.185479,1075.0,98.366018,1282.439375


In [41]:
# Fix the well's TVD as this might be a typo (734, instead of 134)
mask = (main_add.HorizontalDisplacement_hvs<main_add.TotalDepth-main_add.TVD*np.pi/2) & (main_add.WellProfile=='Horizontal')
main_add.loc[mask, 'TVD'] = 734

In [42]:
# TVD_calc from horizontal model
mask1 = main_add.WellProfile.isin(['Horizontal', 'Directional'])
mask2 = main_add.LateralLength_pewh >= main_add.HorizontalDisplacement_hvs
main_add.loc[mask1 & mask2, 'LateralLength_pewh'] = main_add.loc[mask1 & mask2, 'HorizontalDisplacement_hvs']

main_add.loc[mask1, 'BuildRadius_pewh'] = main_add.loc[mask1, 'HorizontalDisplacement_hvs'] - main_add.loc[mask1, 'LateralLength_pewh']
main_add.loc[mask1, 'KOP_pewh'] = main_add.loc[mask1, 'TotalDepth'] - main_add.loc[mask1, 'LateralLength_pewh'] - main_add.loc[mask1, 'BuildRadius_pewh']*np.pi/2
main_add.loc[mask1, 'TVD_calc'] = main_add.loc[mask1, 'KOP_pewh'] + main_add.loc[mask1, 'BuildRadius_pewh']

# New variables from directional model 1
main_add.loc[mask1, 'BuildRadiusRatio_pewh'] = main_add.loc[mask1, 'BuildRadius_pewh']/(main_add.loc[mask1, 'TotalDepth']-main_add.loc[mask1, 'LateralLength_pewh'])
main_add.loc[mask1, 'InclinationLocal_pewh'] = np.arcsin(main_add.loc[mask1, 'BuildRadiusRatio_pewh'])

In [43]:
# TVD_calc for vertical wells
mask = (main_add.WellProfile=='Vertical')
main_add.loc[mask, 'LateralLength_pewh'] = 0
main_add.loc[mask, 'BuildRadius_pewh'] = 0
main_add.loc[mask, 'KOP_pewh'] = main_add.loc[mask, 'TotalDepth']
main_add.loc[mask, 'TVD_calc'] = main_add.loc[mask, 'TotalDepth']

main_add.loc[mask, 'BuildRadiusRatio_pewh'] = 0
main_add.loc[mask, 'InclinationLocal_pewh'] = 0

In [48]:
# Additional ideas
main_add['HorizontalRatio_wh'] = main_add.HorizontalDisplacement_hvs/main_add.TotalDepth
main_add['LateralRatio_pewh'] = main_add.LateralLength_pewh/main_add.TotalDepth

main_add['InclinationTotal_wh'] = np.arcsin(main_add.HorizontalRatio_wh)
main_add['BuildRateTotal_pewh'] = main_add.InclinationTotal_wh/main_add.TotalDepth
main_add['BuildRateLocal_pewh'] = main_add.InclinationLocal_pewh/main_add.TotalDepth

main_add['KBHeight_wh'] = main_add.KBElevation - main_add.GroundElevation
# main_add['TVDSS_calc'] = main_add.TVD_calc - main_add.KBElevation

#### DateTime and Categorical

In [49]:
# Convert datetime to days
timeref = pd.to_datetime('01012000', format='%d%m%Y')
for var in main_add.columns[main_add.dtypes=='datetime64[ns]']: 
    main_add[var+'_day'] = (main_add[var] - timeref).dt.days
    main_add = main_add.drop(var, axis=1)

In [50]:
# Rig Working Time
main_add['RigWorkTime_wh'] = main_add.FinalDrillDate_day - main_add.SpudDate_day
# Rig Idle Time
main_add['RigIdleTime_wh'] = main_add.RigReleaseDate_day - main_add.FinalDrillDate_day
# Well Age Paper
main_add['WellAgePaper_wh'] = main_add.StatusDate_day - main_add.LicenceDate_day
# Well Age Real
main_add['WellAgeReal_wh'] = main_add.StatusDate_day - main_add.SpudDate_day
# Well Condidential Time Paper
main_add['ConfidentialTimePaper_wh'] = main_add.ConfidentialReleaseDate_day - main_add.LicenceDate_day
# Well Condidential Time Real
main_add['ConfidentialTimeReal_wh'] = main_add.ConfidentialReleaseDate_day - main_add.SpudDate_day

In [51]:
# Well Idle Times
main_add['DrillCompleteIdleTime_pewh'] = main_add.CompletionStartDate_perf_day - main_add.FinalDrillDate_day
main_add['CompleteProdIdleTime_pepr'] = main_add.StartDate_prod_day - main_add.CompletionEndDate_perf_day
main_add['TotalIdleTime_pewh'] = main_add.DrillCompleteIdleTime_pewh + main_add.CompleteProdIdleTime_pepr

#### Neighbors Features

In [52]:
# Formation distance matrix
df = main_add['Formation'].factorize()[0].reshape(-1,1)
formation_matrix = cdist(df, df, metric='euclidean')

In [53]:
totaldistance_matrix = distance_matrix + formation_matrix*1000

In [54]:
# train+test x train+test
n_neighbors = 50
neighbors = np.empty((totaldistance_matrix.shape[0], n_neighbors))
for i in range(0, totaldistance_matrix.shape[0]): 
    neighbors[i, :] = list(main_add.EPAssetsId.values[np.argsort(totaldistance_matrix[i, :])[1:n_neighbors+1]])

In [55]:
# Exclude WellHeader variables
varlist = wh.columns.values
varlist = np.append(varlist, ['FFP'])
# Exclude time variables
varlist = np.union1d(varlist, main_add.columns[main_add.columns.str.contains('_day')])
varlist = np.union1d(varlist, main_add.columns[main_add.columns.str.contains('Time')])
varlist = np.union1d(varlist, main_add.columns[main_add.columns.str.contains('Age')])

sigvar = np.setdiff1d(main_add.columns.values, varlist)

# Add some significant original, perf, prod variables
sigvar = np.append(sigvar, ['TVD', 'TotalDepth', 'GroundElevation', 'KBElevation', 'DrillMetresPerDay', 'DaysDrilling', 'ProjectedDepth', 
                            'WellAgeReal_wh', 'WellAgePaper_wh', 'RigIdleTime_wh', 'DrillCompleteIdleTime_pewh', 'CompletionTime_perf'])

impvar = ['TVD', 'TotalDepth', 'GroundElevation', 'KBElevation', 
          'TVD_calc', 'Azimuth_ltln', 'BuildRadius_pewh', 'BuildRateTotal_pewh', 
          'HorizontalDisplacement_hvs', 'KOP_pewh', 'LateralLength_pewh', 
          'WellAgeReal_wh', 'WellAgePaper_wh', 'RigIdleTime_wh', 'DrillCompleteIdleTime_pewh', 
          'CompletionTime_perf', 'CompletionTop_perf', 'nCompletion_perf', 'MaxGas_prod', 'MinGas_prod']

set(sigvar).issuperset(set(impvar))

True

In [57]:
# For significant features (take some time)
for n_neighbors in [5, 10, 15, 20, 25, 30]: 
    for var in sigvar: 
        for index in main_add.index:
            mask = main_add.EPAssetsId.isin(neighbors[index, :n_neighbors])
            main_add.loc[main_add.index == index, var+'_nbmean'+str(n_neighbors)] = main_add.loc[mask, var].mean()
            main_add.loc[main_add.index == index, var+'_nbmin'+str(n_neighbors)] = main_add.loc[mask, var].min()
            main_add.loc[main_add.index == index, var+'_nbmax'+str(n_neighbors)] = main_add.loc[mask, var].max()
        print(var, main_add[var+'_nbmean'+str(n_neighbors)].isnull().sum())
        print(var, main_add[var+'_nbmin'+str(n_neighbors)].isnull().sum())
        print(var, main_add[var+'_nbmax'+str(n_neighbors)].isnull().sum())

AvgCondensate_prod 0
AvgCondensate_prod 0
AvgCondensate_prod 0
AvgGas_prod 0
AvgGas_prod 0
AvgGas_prod 0
AvgOil_prod 0
AvgOil_prod 0
AvgOil_prod 0
AvgWater_prod 0
AvgWater_prod 0
AvgWater_prod 0
Azimuth_ltln 0
Azimuth_ltln 0
Azimuth_ltln 0
BuildRadiusRatio_pewh 0
BuildRadiusRatio_pewh 0
BuildRadiusRatio_pewh 0
BuildRadius_pewh 0
BuildRadius_pewh 0
BuildRadius_pewh 0
BuildRateLocal_pewh 0
BuildRateLocal_pewh 0
BuildRateLocal_pewh 0
BuildRateTotal_pewh 0
BuildRateTotal_pewh 0
BuildRateTotal_pewh 0
CompletionEnd_perf 0
CompletionEnd_perf 0
CompletionEnd_perf 0
CompletionTop_perf 0
CompletionTop_perf 0
CompletionTop_perf 0
CumCondensate_prod 0
CumCondensate_prod 0
CumCondensate_prod 0
CumDuration_prod 0
CumDuration_prod 0
CumDuration_prod 0
CumGas_prod 0
CumGas_prod 0
CumGas_prod 0
CumOil_prod 0
CumOil_prod 0
CumOil_prod 0
CumProdPeriod_prod 0
CumProdPeriod_prod 0
CumProdPeriod_prod 0
CumWater_prod 0
CumWater_prod 0
CumWater_prod 0
HorizontalDisplacement_hvs 0
HorizontalDisplacement_hvs 0


BuildRateTotal_pewh 0
BuildRateTotal_pewh 0
BuildRateTotal_pewh 0
CompletionEnd_perf 0
CompletionEnd_perf 0
CompletionEnd_perf 0
CompletionTop_perf 0
CompletionTop_perf 0
CompletionTop_perf 0
CumCondensate_prod 0
CumCondensate_prod 0
CumCondensate_prod 0
CumDuration_prod 0
CumDuration_prod 0
CumDuration_prod 0
CumGas_prod 0
CumGas_prod 0
CumGas_prod 0
CumOil_prod 0
CumOil_prod 0
CumOil_prod 0
CumProdPeriod_prod 0
CumProdPeriod_prod 0
CumProdPeriod_prod 0
CumWater_prod 0
CumWater_prod 0
CumWater_prod 0
HorizontalDisplacement_hvs 0
HorizontalDisplacement_hvs 0
HorizontalDisplacement_hvs 0
HorizontalRatio_wh 0
HorizontalRatio_wh 0
HorizontalRatio_wh 0
InclinationLocal_pewh 0
InclinationLocal_pewh 0
InclinationLocal_pewh 0
InclinationTotal_wh 0
InclinationTotal_wh 0
InclinationTotal_wh 0
KBHeight_wh 0
KBHeight_wh 0
KBHeight_wh 0
KOP_pewh 0
KOP_pewh 0
KOP_pewh 0
LateralLength_pewh 0
LateralLength_pewh 0
LateralLength_pewh 0
LateralRatio_pewh 0
LateralRatio_pewh 0
LateralRatio_pewh 0
MaxCond

In [62]:
# fillna low n_neighbors features with higher n_neighbors features
for m, n in zip(['5'], ['10']): 
    for var in main_add.columns[main_add.columns.str.endswith(m)]: 
        if main_add[var].isnull().sum() > 0: 
            main_add[var] = main_add[var].fillna(main_add[var[:-1]+n])
            print(var, main_add[var].isnull().sum())    

TVD_nbmean5 0
TVD_nbmin5 0
TVD_nbmax5 0


#### Export

In [70]:
# Check NaN
datatype = pd.DataFrame(main_add.dtypes, columns=['pdtype'])
datatype['nunique'] = main_add.nunique()
datatype['nnan'] = main_add.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8689,0
Province,object,2,0
LicenceNumber,object,8682,0
UWI,object,8689,0
CurrentOperator,object,106,0
...,...,...,...
DrillCompleteIdleTime_pewh_nbmin30,float64,264,0
DrillCompleteIdleTime_pewh_nbmax30,float64,309,0
CompletionTime_perf_nbmean30,float64,3934,0
CompletionTime_perf_nbmin30,float64,60,0


In [71]:
main_add = main_add.drop(['UnitFlag', 'UnitName', 'UnitID', 'Municipality', 'StatusSource', 'RegulatoryAgency'], axis=1)

In [72]:
main_add.shape

(8689, 971)

In [73]:
main_add.to_csv('Xy_Neigh10Add10Weak2.csv', index=False)